To run this you need to do the following:


1.   Create a Spotify Developer account (developer.spotify.com)
2.   Once logged into Spotify Developer portal, go to Dashboard > Create App

3.   In the app settings in the Spotify portal, make the redirect URL https://example.com/callback
4.   Insert your Client ID and Client Secret into the script's placeholder variables




In [ ]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json

#Spotify API credentials
CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'
REDIRECT_URI = 'https://example.com/callback'  #You can use any valid redirect URI
SCOPE = 'playlist-modify-public playlist-modify-private'

#Initialize Spotipy
sp_oauth = SpotifyOAuth(client_id=CLIENT_ID,
                        client_secret=CLIENT_SECRET,
                        redirect_uri=REDIRECT_URI,
                        scope=SCOPE)

#Get the authorization URL
auth_url = sp_oauth.get_authorize_url()
print(f"Please go to this URL and authorize access: {auth_url}")

In [ ]:
token_info = sp_oauth.get_cached_token()

if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect URL here: ')
    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)
    print('Token obtained.')

In [ ]:
#Function to get all playlists
def get_all_playlists():
    playlists = []
    results = sp.current_user_playlists()
    while results:
        playlists.extend(results['items'])
        results = sp.next(results) if results['next'] else None
    return playlists

#Function to get tracks in a playlist
def get_playlist_tracks(playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    while results:
        tracks.extend(results['items'])
        results = sp.next(results) if results['next'] else None
    return tracks

#Function to find drake songs and replace with Kendrick Lamar's "Not like us"
def replace_drake_songs():
    playlists = get_all_playlists()
    kendrick_track_uri = 'spotify:track:6AI3ezQ4o3HUoP6Dhudph3'  # URI for "Not like us" by Kendrick Lamar

    for playlist in playlists:
        playlist_id = playlist['id']
        tracks = get_playlist_tracks(playlist_id)
        for item in tracks:
            track = item['track']
            if track is not None and 'drake' in [artist['name'].lower() for artist in track['artists'] if artist['name']]:
                track_uri = track['uri']
                try:
                    sp.playlist_remove_all_occurrences_of_items(playlist_id, [track_uri])
                    sp.playlist_add_items(playlist_id, [kendrick_track_uri])
                    print(f"Replaced {track['name']} by Drake with Kendrick Lamar's 'Not like us' in playlist {playlist['name']}")
                except spotipy.SpotifyException as e:
                    if e.http_status == 403:
                        print(f"Skipping playlist {playlist['name']} due to insufficient permissions.")
                    else:
                        raise

#Execute the replacement
replace_drake_songs()